In [1]:
import numpy as np
import random 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy 

In [2]:
hidden_layers=[16,16]    #[2,3,2] three layers respectively with 2, 3, 2 neurons

weights=[]               #this is a list of a few matrices [weight matrices]
biases=[]                #this is a list of vectors [biases]

activations=[-1]*(len(hidden_layers)+2)    
                         #keeps track of activations at 
                         #each layer (after activated by activation func)
num_of_inputs=784                     # def = 784  (MNIST)
num_of_outputs=10                      # def = 10  (MNIST)
alpha=0.01                                  #learning_rate
batch_size=500

#generate biases for layer 2,3,....output
hidden_layers= hidden_layers + [num_of_outputs]

for i in range (len(hidden_layers)):
    temp=[]
    for k in range (hidden_layers[i]):
        temp.append(random.uniform(-0.5,+0.5))
        #temp.append(random.uniform(0,1))#/2)
    biases.append(np.array(temp))

#10 output for MNIST dataset    
#generate random weights inside the neural 
hidden_layers=[num_of_inputs] + hidden_layers        

for t in range(len(hidden_layers)-1):
    temp=[]
    for q in range (hidden_layers[t]*hidden_layers[t+1]):
        temp.append(random.uniform(-0.5,+0.5))
        #temp.append(random.uniform(0,1))#/2)
        
    matrix_temp=np.array(temp)
    matrix_temp=matrix_temp.reshape((hidden_layers[t],hidden_layers[t+1]))
    weights.append(matrix_temp)

#print("weights:\n", weights)
#print("biases:\n", biases)


#functions#############################################################################
def sigmoid(x):                  # x=numpy so it affects all elements
    return 1/(1+np.exp(-x))



def  feed_forward (input): #input np.array SHAPE=(1,784)
    activations[0]=input
    for t in range (len(weights)):
        next = np.dot(input,weights[t])
        next = next + biases[t]
        next = sigmoid (next)
        activations[t+1]=next   
        input=next
    return input

def feed_backward (t_minus_y,alpha): #t_minus_y is a 1x10 numpy giving errors
    
    change_in_weights=[] #new list of matrices each element showing the delta_W
    change_in_biases=[] #new list of matrices each element showing the delta_B
      
    
    f_of_y_in=activations[-1]
    delta_k_all= t_minus_y * f_of_y_in * (1 - f_of_y_in)

    change_in_biases.append(alpha * delta_k_all)
    
    for k in range (2,len(hidden_layers)+1):
        z = activations[-k].reshape(1,-1)
    
    #z = activations[-2].reshape(1,-1)
        change_in_weights.append(alpha * np.dot(z.T,delta_k_all.reshape(1,-1)) )    
        delta_k_in_all= np.dot(delta_k_all.reshape(1,-1),weights[-k+1].T)
        f_of_z_in=activations[-k]
   #     f_of_z_in=activations[-2]
        delta_k_all= delta_k_in_all * f_of_z_in * (1- f_of_z_in)
   #     delta_j_all= delta_j_in_all * f_of_z_in * (1- f_of_z_in)
    
        if k<len(hidden_layers):
            change_in_biases.append(alpha * delta_k_all)
   #     z = activations[-3].reshape(1,-1)
   #     change_in_weights.append(alpha * np.dot(z.T,delta_j_all.reshape(1,-1)) )
        
   # print("weights change at this step:", change_in_weights)
    
    return change_in_weights, change_in_biases


def weight_bias_updater (tuple):  #(weights_changes, biases_changes)
    w=tuple[0]
    w.reverse()
    b=tuple[1]
    b.reverse()
    
    for t in range (len(b)):
        biases[t] = biases[t]+b[t]
        weights[t]= weights[t]+ w[t]      

def MSE(target, output):
    return np.average((target - output)**2)

def One_Hot_Encoding(int):
    if int==0:           return np.array([1,0,0,0,0,0,0,0,0,0])
    elif int==1:         return np.array([0,1,0,0,0,0,0,0,0,0])
    elif int==2:         return np.array([0,0,1,0,0,0,0,0,0,0]) 
    elif int==3:         return np.array([0,0,0,1,0,0,0,0,0,0])
    elif int==4:         return np.array([0,0,0,0,1,0,0,0,0,0])
    elif int==5:         return np.array([0,0,0,0,0,1,0,0,0,0])  
    elif int==6:         return np.array([0,0,0,0,0,0,1,0,0,0])
    elif int==7:         return np.array([0,0,0,0,0,0,0,1,0,0])
    elif int==8:         return np.array([0,0,0,0,0,0,0,0,1,0])
    elif int==9:         return np.array([0,0,0,0,0,0,0,0,0,1])
    
def min_max_scale (array):
    min=np.min(array)
    max=np.max(array)
    denom= max - min
    if max==0:
        return (np.array([0]*len(array)))
    else:
        return (array - min)/denom
  

def batch_adder (tup1, tup2):
  w1 = tup1[0]
  w2 = tup2[0]
  b1 = tup1 [1]
  b2 = tup2 [1]
  w3=[] 
  b3=[]

  for i in range (len(w1)):
    w3.append(w1[i]+w2[i])
    b3.append(b1[i]+b2[i])

  return (w3,b3)

def batch_list_adder (list):
  my_list=copy.copy(list)
  for s in range (len(my_list)-1):
       my_list[s+1]=batch_adder(my_list[s],my_list[s+1])

  return my_list[-1]


In [3]:
df_train=pd.read_csv('mnist_train.csv') 
#df_train=pd.read_csv('mnist_train.csv')[0:100]

df_train_labels= df_train['label'].values
df_train_labels2=np.array([]) #one hot encoded
for i in range (len(df_train_labels)):
    df_train_labels2 = np.append (df_train_labels2, One_Hot_Encoding(df_train_labels[i]))
    
#df_train_labels3 = df_train_labels2.reshape(60000,10)   
df_train_labels3 = df_train_labels2.reshape(60000,10) 

df_train_features = df_train.drop(columns=["label"])
df_train_features = df_train_features.values

In [4]:
#feature scaling
df_train_features_scaled=np.array([])
for t in range (784):
    df_train_features_scaled=np.append(df_train_features_scaled,min_max_scale(df_train_features[:,t]))

df_train_features_scaled = df_train_features_scaled.reshape(784,60000).T

In [4]:
alpha=0.001
batch_size=500
for i in range (2):  #epochs
    sum=0
    for t in range(0,60000,batch_size):
        list_of_change_tuples=[]
        #print('new_batch starting ... \n:')
        for b in range(batch_size):
          #print(t+b)
          input=  df_train_features[t+b]
          #print(train_features[t], train_labels[t])
          output= feed_forward(input)
          error = df_train_labels3[t+b] - output
          change_tuple = feed_backward(error,alpha)
          list_of_change_tuples.append(change_tuple)
          #print(change_tuple)
          sum+=MSE(df_train_labels3[t+b],output)
            

        all_changes= batch_list_adder (list_of_change_tuples)
        weight_bias_updater(all_changes)

    print('epoch ', i, 'completed . . . squared error: ', sum)

<ipython-input-2-06aa0043dc3d>:44: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


epoch  0 completed . . . squared error:  5768.579565772131
epoch  1 completed . . . squared error:  5332.489361532528


In [11]:
acc=0
for t in range (60000):
    if np.argmax(feed_forward(df_train_features[t]))== df_train_labels[t]:
        acc+=1
print("acc = ", acc/60000)

<ipython-input-2-06aa0043dc3d>:44: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


acc =  0.8817666666666667


In [32]:
np.argmax(feed_forward(df_train_features[50000]))

3

In [42]:
np.argmax(feed_forward(df_train_features[30000]))

5

In [12]:
df_test=pd.read_csv('mnist_test.csv') 
#df_train=pd.read_csv('mnist_train.csv')[0:100]

df_test_labels= df_test['label'].values
df_test_features = df_test.drop(columns=["label"])
df_test_features = df_test_features.values

In [13]:
acc=0
for t in range (10000):
    if np.argmax( feed_forward(df_test_features[t])      )== df_test_labels[t]:
        acc+=1
print("acc = ", acc/10000)

<ipython-input-2-06aa0043dc3d>:44: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


acc =  0.8809


<ipython-input-45-401f473addbc>:42: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


9

In [1]:
#r=3719
#c=df_test.loc[r].values
#rl=c[0]
#c=np.delete(c, [0])
#p=c.reshape((28,28))
#plt.imshow(p, cmap='gray')
#plt.show()
#print(np.argmax(feed_forward(df_test_features[r])))
#print(df_test_labels[r])

In [2]:
#acc=0
#for t in range (10000):
 #   if np.argmax( feed_forward(df_test_features[t])      )!= df_test_labels[t]:
#        rl=df_test_labels[t]
#        c=df_test_features[t]
#        p=c.reshape((28,28))
#        plt.imshow(p, cmap='gray')
#        plt.show()
 #       print("NN output: ", np.argmax(feed_forward(df_test_features[t])))
 #       print("Label    : ", df_test_labels[t])